# Monitoramento de Ações Via Telegram

## Install Dependencies

In [0]:
!pip install requests
!pip install yfinance

## Imports

In [0]:
import yfinance as yf
import pandas as pd
import requests
import json
from datetime import datetime

## Bot Configuration

In [0]:
# URLs do Telegram Bot
# https://api.telegram.org/bot<SEU_TOKEN>/getUpdates

url = "https://api.telegram.org/bot<SEU_TOKEN>/sendMessage"

config = [{
    "acao" : "",
    "percent_caiu" : 1,
    "percent_subiu" : 1.02,
    "vl_caiu" : 0.15,
    "vl_subiu" : 0.15,
    "ids_chat" : [<SEU_CHAT_ID>]
    },       
    {
    "acao" : "",
    "percent_caiu" : 1,
    "percent_subiu" : 1.02,
    "vl_caiu" : 0.15,
    "vl_subiu" : 0.15,
    "ids_chat" : [<SEU_CHAT_ID>]
    }          
]

## Debug Logging

In [0]:
def log_debug(msg):
  """
  Exibe mensagens de debug com timestamp.
    
  """

  print(f"[DEBUG] {datetime.now()} - {msg}")


## Get Stock Prices

In [0]:
def get_ticket(p_acao):
    '''
    Retorna os valores de uma determinada ação na última cotação disponível.

    Args: 
        p_acao::String
    Returns:        
        vl_abertura::Float
        vl_fechamento::Float
        vl_variacao::Float
        percent_variacao::Float
    '''
    
    try:
        df_cotacao = yf.download(p_acao, period='1d')
        vl_abertura = df_cotacao.iloc[0]['Open']
        vl_fechamento = df_cotacao.iloc[0]['Close']

        # TRATA ERRO NO YFINANCE
        if vl_abertura == 0 or vl_fechamento == 0:
            log_debug(f"Problema ao obter dados de {p_acao}!")
            df_cotacao = yf.download(p_acao, period='2d')
            vl_abertura = df_cotacao.iloc[0]['Open']
            vl_fechamento = df_cotacao.iloc[0]['Close']

        vl_variacao = vl_fechamento - vl_abertura    
        percent_variacao = ((vl_fechamento * 100) / vl_abertura) - 100 

        return {
            "vl_abertura": round(vl_abertura, 2),
            "vl_fechamento": round(vl_fechamento, 2),
            "vl_variacao": round(vl_variacao, 2),
            "percent_variacao": round(percent_variacao, 2)
        }
    except Exception as e:
        print(f"[Erro] Erro ao buscar cotação de {p_acao}. Detalhes: {e}")
        return None
    

## Send Message

In [0]:
def send_message_telegram(p_idChat, p_text):
    '''
    Envia mensagem ao Telegram via BOT

    Args:
        p_idChat::List -> Código único da conversa
        p_text::String -> Corpo da mensagem
    '''

    for id_chat in p_idChat:
        payload = json.dumps({
            "chat_id": id_chat,
            "text": p_text,
            "parse_mode": "HTML"
        })

        headers = {
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

    return True


## Alert Verification

In [0]:
def verificar_alertas(acao, trigger_alerta, json_cotacao): 
    """
    Verifica se os valores da ação disparam os gatilhos de alerta definidos.

    """

    real_vl_variacao = json_cotacao.get('vl_variacao', 0)
    real_percent_variacao = json_cotacao.get('percent_variacao', 0)
    trigger_ids_chat = trigger_alerta.get("ids_chat", [])

    # VALIDA SE A AÇÃO SUBIU EM %        
    if trigger_alerta.get("percent_subiu", 0) != 0:
        log_debug(f'Alerta se {acao} subir {trigger_alerta.get("percent_subiu", 0)} %')
        log_debug(f'{acao} variou {real_percent_variacao} %')
        
        if trigger_alerta.get("percent_subiu", 0) <= real_percent_variacao:
            log_debug('--->>> GATILHO ATIVADO! <<<---')

            mensagem = (
                f"📈 <b>ALERTA!</b>\n\n"
                f"<i>Monitor de Ações BOT:</i>\n"
                f"<b>Ação:</b> {acao}\n"            
                f"<b>Gatilho:</b> {trigger_alerta.get('percent_subiu', 0)}% de alta\n"
                f"<b>Variação:</b> {real_percent_variacao}% (Alta)\n"
            )

            send_message_telegram(trigger_ids_chat, mensagem)

        print("--------------------------------------------------------------------")    

    # VALIDA SE A AÇÃO CAIU EM %
    if trigger_alerta.get("percent_caiu", 0) != 0:
        log_debug(f'Alerta se {acao} cair {trigger_alerta.get("percent_caiu", 0)} %')
        log_debug(f'{acao} variou {real_percent_variacao} %')   

        if trigger_alerta.get("percent_caiu", 0) * -1 >= real_percent_variacao:
            log_debug('--->>> GATILHO ATIVADO! <<<---')

            mensagem = (
                f"⚠️ <b>ALERTA!</b>\n\n"
                f"<i>Monitor de Ações BOT:</i>\n"
                f"<b>Ação:</b> {acao}\n"
                f"<b>Gatilho:</b> {trigger_alerta.get('percent_caiu', 0)}% de baixa\n"
                f"<b>Variação:</b> {real_percent_variacao}% (Baixa)\n"
            )

            send_message_telegram(trigger_ids_chat, mensagem)

        print("--------------------------------------------------------------------")

    # VALIDA SE A AÇÃO SUBIU EM VALOR
    if trigger_alerta.get("vl_subiu", 0) != 0:
        log_debug(f'Alerta se {acao} subir R$ {trigger_alerta.get("vl_subiu", 0)}')
        log_debug(f'{acao} variou {real_vl_variacao} %')

        if trigger_alerta.get("vl_subiu", 0) <= real_vl_variacao:
            log_debug('--->>> GATILHO ATIVADO! <<<---')

            mensagem = (
                f"📈 <b>ALERTA!</b>\n\n"
                f"<i>Monitor de Ações BOT:</i>\n"
                f"<b>Ação:</b> {acao}\n"
                f"<b>Gatilho:</b> R$ {trigger_alerta.get('vl_subiu', 0)} de alta\n"
                f"<b>Variação:</b> R$ {real_vl_variacao} (Alta)\n"
            )
            send_message_telegram(trigger_ids_chat, mensagem)
        
        print("--------------------------------------------------------------------")

    # VALIDA SE A AÇÃO CAIU EM VALOR
    if trigger_alerta.get("vl_caiu", 0) != 0:
        log_debug(f'Alerta se {acao} cair R$ {trigger_alerta.get("vl_caiu", 0)}')
        log_debug(f'{acao} variou {real_vl_variacao} %')

        if trigger_alerta.get("vl_caiu", 0) * -1 >= real_vl_variacao:
            log_debug('--->>> GATILHO ATIVADO! <<<---')

            mensagem = (
                f"📉 <b>ALERTA!</b>\n\n"
                f"<i>Monitor de Ações BOT:</i>\n"
                f"<b>Ação:</b> {acao}\n"
                f"<b>Gatilho:</b> R$ {trigger_alerta.get('vl_caiu', 0)} de baixa\n"
                f"<b>Variação:</b> R$ {real_vl_variacao} (Baixa)\n"
            )
            send_message_telegram(trigger_ids_chat, mensagem)
        
        print("--------------------------------------------------------------------")


## Run

In [0]:
for trigger_alerta in config:
    log_debug(f"Processando gatilhos para a ação: {trigger_alerta['acao']}")
    
    acao = trigger_alerta.get("acao", "")
    json_cotacao = get_ticket(acao)
    
    if json_cotacao:
        verificar_alertas(acao, trigger_alerta, json_cotacao)
    else:
        log_debug(f"Não foi possível obter dados para a ação {acao}.")
    
    print("####################################################################")


[DEBUG] 2024-11-02 00:49:38.485391 - Processando gatilhos para a ação: BBAS3.SA
[*********************100%%**********************]  1 of 1 completed
[DEBUG] 2024-11-02 00:49:38.573453 - Alerta se BBAS3.SA subir 1.02 %
[DEBUG] 2024-11-02 00:49:38.573479 - BBAS3.SA variou -0.99 %
--------------------------------------------------------------------
[DEBUG] 2024-11-02 00:49:38.573555 - Alerta se BBAS3.SA cair 1 %
[DEBUG] 2024-11-02 00:49:38.573566 - BBAS3.SA variou -0.99 %
--------------------------------------------------------------------
[DEBUG] 2024-11-02 00:49:38.573580 - Alerta se BBAS3.SA subir R$ 0.15
[DEBUG] 2024-11-02 00:49:38.573588 - BBAS3.SA variou -0.26 %
--------------------------------------------------------------------
[DEBUG] 2024-11-02 00:49:38.573600 - Alerta se BBAS3.SA cair R$ 0.15
[DEBUG] 2024-11-02 00:49:38.573608 - BBAS3.SA variou -0.26 %
[DEBUG] 2024-11-02 00:49:38.573615 - --->>> GATILHO ATIVADO! <<<---
-----------------------------------------------------------